<a href="https://colab.research.google.com/github/Arif111866/Parallel-Processing-Lab-4.1/blob/main/Cuda_matrixmul_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

X, A, B
X => C1 = A1 * B1
C2 = A2 * B2

In [1]:
%%writefile arif.cu
#include <iostream>
#include <cuda_runtime.h>
using namespace std;

__global__ void matrixMul(float* A, float* B, float* C, int M, int N, int P, int offset) {
    int k = threadIdx.x + offset;

    float* a = A + k * M * N;
    float* b = B + k * N * P;
    float* c = C + k * M * P;

    for(int i = 0; i < M; i++) {
        for(int j = 0; j < N; j++) {
            for(int l = 0; l < P; l++) {
                //c[i][l] += a[i][j] * b[j][l];
                c[i * P + l] += a[i * N + j] * b[j * P + l];
            }
        }
    }
}

int main(int argc, char *argv[]) {

    int T = atoi(argv[1]); //koyta thread use korte parbo
    int K = atoi(argv[2]); //koita matrix gun

    //100 gun, thread 10,

    int M = 400, N = 400, P = 400;

    int SizeA = M * N * K;
    int SizeB = N * P * K;
    int SizeC = M * P * K;

    //memory alocate (cpu allocate)
    float *h_A = new float[SizeA];
    float *h_B = new float[SizeB];
    float *h_C = new float[SizeC];


    //malloc (gpu allocate)
    float *d_A;
    cudaMalloc(&d_A, SizeA * sizeof(float));
    float *d_B;
    cudaMalloc(&d_B, SizeB * sizeof(float));
    float *d_C;
    cudaMalloc(&d_C, SizeC * sizeof(float));

    //data initialize
    for (int i = 0; i < SizeA; i++) {
        h_A[i] = rand();
    }
    for(int i = 0; i < SizeB; i++) {
        h_B[i] = rand();
    }


    //copy from host to device
    cudaMemcpy(d_A, h_A, SizeA * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, SizeB * sizeof(float), cudaMemcpyHostToDevice);

    //cuda process suru
    int gunKorteHobe = K;
    int offset = 0;
    while(gunKorteHobe > 0){

        int currentBatch = min(gunKorteHobe, T);

        matrixMul<<<1,currentBatch>>>(d_A, d_B, d_C, M, N, P, offset);
        cudaDeviceSynchronize();

        gunKorteHobe -= currentBatch;
        offset += currentBatch;
    }

    //let's copy back to cpu
    cudaMemcpy(h_C, d_C, SizeC * sizeof(float), cudaMemcpyDeviceToHost);

    cout << "All operation done" << endl;

}

Writing asif.cu


In [2]:
!nvcc -arch=sm_75 arif.cu -o arif

In [3]:
!time ./arif 1 10 && sleep 2

All operation done

real	1m1.000s
user	0m59.769s
sys	0m0.255s


In [4]:
!time ./arif 3 10 && sleep 2

All operation done

real	0m25.212s
user	0m24.934s
sys	0m0.143s


In [5]:
!time ./arif 7 10 && sleep 2

All operation done

real	0m13.946s
user	0m13.661s
sys	0m0.127s


In [6]:
!time ./arif 10 10 && sleep 2

All operation done

real	0m8.077s
user	0m7.943s
sys	0m0.117s


In [7]:
!time ./arif 20 10 && sleep 2

All operation done

real	0m8.114s
user	0m7.948s
sys	0m0.139s
